From Scratch implementation of a Linear network


In [ ]:
#import dependencies
import numpy as np
from scipy import optimize
import os
from PIL import Image
import random

In [ ]:
#import dataset
!git clone https://github.com/digitalepidemiologylab/plantvillage_deeplearning_paper_dataset.git

Cloning into 'plantvillage_deeplearning_paper_dataset'...
remote: Enumerating objects: 163224, done.
remote: Total 163224 (delta 0), reused 0 (delta 0), pack-reused 163224
Receiving objects: 100% (163224/163224), 2.00 GiB | 32.84 MiB/s, done.
Resolving deltas: 100% (97/97), done.
Checking out files: 100% (182401/182401), done.


In [ ]:
#Find locations of relevant files in dataset
directory = '/content/plantvillage_deeplearning_paper_dataset/raw/color'
healthy_dir = []
unhealthy_dir = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    #if os.path.isfile(f):
    #if 'Apple' in str(f):
    if 'Apple' in str(f):
      if 'healthy' in str(f):
        healthy_dir.append(f)
      else:
        unhealthy_dir.append(f)

In [ ]:
#Open images, resize them and flatten them
healthy = []
unhealthy = []
for dir in healthy_dir:
  for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    healthy.append(np.array(Image.open(f).resize((16,16)))[:,:,:3].flatten()/256)
for dir in unhealthy_dir:
  for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    unhealthy.append(np.array(Image.open(f).resize((16,16)))[:,:,:3].flatten()/256)

In [ ]:
#Create labels
train = np.array(healthy + unhealthy)
print(train.shape)
labels = np.array([[1,0] for i in range(len(healthy))] + [[0,1] for i in range(len(unhealthy))])
print(labels.shape)

(3171, 768)
(3171, 2)


In [ ]:
#Class for Linear Layer objects
class LinearLayer:
  def __init__(self, in_ft, out_ft, input = False, output = False):
    bound = np.sqrt(1/in_ft) #boundraries calculated for initialization
    self.weights = np.random.uniform(low = -bound, high = bound, size = (in_ft,out_ft)) #Weights according to initialization
    self.bias = np.random.uniform(low = -bound, high = bound, size = (1,out_ft)) #Bias according to initialization
    self.input = input 
    self.output = output
  
  def call(self, input): # run a forward pass of the Layer and retain gradients
    def compute(weights,input):
      return (input @ weights) + np.stack(input.shape[0] * [self.bias]).reshape(input.shape[0], -1 )
    weights = self.weights
    def get_partials(weights,input):
      return np.stack(input.shape[0] *[np.ones_like(weights)]) * input.reshape(-1,input.shape[1],1)
    self.partials = get_partials(weights,input)
    self.bpartials = np.stack(input.shape[0]* [np.identity(weights.shape[1])])
    self.forwardgrad = np.stack([weights] * input.shape[0])
    return compute(weights, input)
  def is_in(self):
    return self.input
  def is_out(self):
    return self.output
  def update(self, delta):
    self.weights += delta
  def backward(self, retain, lr): #Update a layer given the gradient of the following layers and the learning rate
    self.weights += lr * np.sum(self.partials * retain.transpose(0,2,1), 0)
    self.bias += lr * np.sum(retain.transpose(0,2,1) @ self.bpartials, 0)
    return self.forwardgrad  @ retain
    

In [ ]:
class ReLU:
  def __init__(self, input = False, output = False):
    self.input = input
    self.output = output
  def call(self, input):
    self.forwardgrad = np.stack([np.diag(np.where(i>0, 1, 0)) for i in input])
    self.partials = np.maximum(0,input)
    return self.partials
  def is_in(self):
    return self.input
  def is_out(self):
    return self.output
  def backward(self, retain, lr):
    return self.forwardgrad  @ retain

In [ ]:
class MSE:
  def loss(self, y, y_p):
    return np.sum((y_p - y) ** 2)/len(y)
  def grad(self, y, y_p):
    return -2*(y_p - y) * y

In [ ]:
class simple_NN:
  def __init__(self):
    self.layers = []
    self.layers.append(LinearLayer(10,5))
    self.layers.append(LinearLayer(5,1))

  def forward(self, x):
    for layer in self.layers:
      x = layer.call(x)
    return x
  def backward(self, lgrad, lr):
    retain = np.array(lgrad).reshape(self.layers[0].partials.shape[0],-1,1)
    for layer in reversed(self.layers):
      retain = layer.backward(retain, lr)


In [ ]:
simple_net = simple_NN()

In [ ]:
criterion = MSE()

In [ ]:
for i in range(10):
  X = np.random.uniform(size = (20, 10))
  y_pred = simple_net.forward(X)
  y = np.sum(X, axis =1).reshape(20,1) * 0.5
  print(criterion.loss(y_pred, y))
  #print(net.layers[0].weights)
  #print(y.shape)
  #print(labels[:10].shape)
  simple_net.backward(criterion.grad(y, y_pred), 0.0001)

0.06542612387437993
0.05830847228989043
0.060884621649259785
0.0739705833619717
0.060121448785648235
0.0751634860228629
0.07466104553771002
0.062185182951090566
0.06403638563305554
0.062170310958654953


In [ ]:
class NN:
  def __init__(self):
    self.layers = []
    self.layers.append(LinearLayer(768,64, input = True))
    #self.layers.append(ReLU())
    self.layers.append(LinearLayer(64,32))
    #self.layers.append(ReLU())
    self.layers.append(LinearLayer(32,2))
    self.layers.append(ReLU(output = True))
    #self.layers.append(LinearLayer(3,2))
    #self.layers.append(LinearLayer(2,12, output = True))
  def forward(self, x):
    for layer in self.layers:
      x = layer.call(x)
    return x
  def backward(self, lgrad, lr):
    retain = np.array(lgrad).reshape(self.layers[0].partials.shape[0],-1,1)
    for layer in reversed(self.layers):
      retain = layer.backward(retain, lr)




In [ ]:
net = NN()

In [ ]:
criterion = MSE()

In [ ]:
class ImageLoader():
    def __init__(self, dset, labels, batch_size):
      indices = list(range(dset.shape[0]))
      random.shuffle(indices)
      #print(indices)
      self.dset = dset[indices]
      self.labels = labels[indices]
      self.calls = self.dset.shape[0] // batch_size
    def call(self,i):
      return (self.dset[i*10:(i+1)*10], self.labels[i*10:(i+1)*10])
      

In [ ]:
train_loader = ImageLoader(train, labels, 100)

In [ ]:
print(train_loader.calls)

31


In [ ]:
for i in range(train_loader.calls):
  X, y = train_loader.call(i)
  y_pred = net.forward(X)
  print(criterion.loss(y_pred, y))
  net.backward(criterion.grad(y, y_pred), 0.001)

0.8244601731153696
0.49533015743901193
0.6717016587963345
0.617846315440259
0.6272463840849242
0.6602934952297299
0.8194606935410356
0.9658575694167553
0.8600052858309688
1.007164879131294
0.9582632012509584
1.0340887398336167
0.9634847039085702
0.9937875268873597
1.1056646947094246
1.0403588756183206
0.888970024713549
0.9324852659780648
0.9936944959054521
1.2382414018923749
0.9478560845205456
0.9262562235467268
1.06504744894536
1.1024533171710953
1.075870065405121
0.9870335022269083
0.9563484167271321
0.9249361594509043
1.00678458741229
1.0961091262281957
0.8666579185024623
